<a href="https://colab.research.google.com/github/jackie-Gung/Colab_ESAA/blob/main/2022_11_14_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Chapter 9. 추천 시스템**
---

### **05. 콘텐츠 기반 필터링 실습**
---

#### **1. 장르 속성을 이용한 콘텐츠 기반 필터링**

- **개요**
  - **개념:** 사용자가 좋아한다고 감상한 영화와 비슷한 특성과 속성, 구성 요소 등을 가진 다른 영화를 추천하는 것
  - **중요한 요소:** 영화 장르 속성

#### **2. 데이터 로딩 및 가공**

In [2]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

In [3]:
movies = pd.read_csv('/content/drive/MyDrive/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


- **분석:** 4803개의 레코드와 20개의 피처로 구성됨

In [4]:
movies_df = movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]

- **주의해야 할 칼럼**
  - **`genres`**, **`keywords`**: 리스트 내부에 여러 딕셔너리 형태의 문자열이 표기돼 있음 -> 가공 필수

In [6]:
pd.set_option('max_colwidth',100)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


- ast 모듈의 **`literal_eval()`**
  - 문자열을 문자열이 의미하는 **list[dist1,dist2]** 객체로 만들어줌

In [7]:
from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [8]:
movies_df['genres'] = movies_df['genres'].apply(lambda x: [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x: [y['name'] for y in x])
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


#### **3. 장르 콘텐츠 유사도 측정**

- **장르별 유사도 측정 방법**
  - 영화들끼리 겹치는 장르가 있을 때
  - genres를 문자열로 변경
  - countvectorizer로 피처 벡터화
  - 행렬 데이터 값을 코사인 유사도로 비교
  - 장르 유사도가 높은 영화 중 평점이 높은 순으로 영화 추천

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer 적용을 위해 공백문자로 word 단위가 구분되는 문자열로 변환하기
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x: (' ').join(x))
count_vect = CountVectorizer(min_df=0,ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal']) # 피처 벡터화한 행렬
print(genre_mat.shape)

(4803, 276)


In [12]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat,genre_mat)
print(genre_sim.shape)
print(genre_sim[:1])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]


- 콘텐츠 기반 필터링 수행
  - movies_df 개별 레코드에 대해 가장 장르 유사도가 높은 순으로 다른 레코드 추출
  - 앞에서 생성한 genre_sim 객체 이용
  - 기준이 되는 행과의 유사도가 높은 순으로 정렬된 행렬의 **위치 인덱스 값** 추출

In [13]:
genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


#### **4. 장르 콘첸츠 필터링을 이용한 영화 추천**

- **장르 유사도에 따라 영화를 추천하는 함수 생성하기**

In [14]:
def find_sim_movie(df,sorted_ind,title_name,top_n=10):
  # movies_df에서 title 칼럼이 입력된 title_name 값인 데이터프레임 추출하기
  title_movie = df[df['title'] == title_name]

  # title_name을 가진 인덱스 객체를 ndarray로 반환하기
  # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출하기
  title_index = title_movie.index.values
  similar_indexes = sorted_ind[title_index,:(top_n)]

  # 추출된 top_n index 출력하기
  print(similar_indexes)
  similar_indexes = similar_indexes.reshape(-1)

  return df.iloc[similar_indexes]

In [15]:
similar_movies = find_sim_movie(movies_df,genre_sim_sorted_ind,'The Godfather',10)

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


In [16]:
similar_movies[['title','vote_average']]

,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


- **분석:** 대부II처럼 비슷한 장르도 있지만 Light Sleeper나 Mi America처럼 평점이 낮은 장르도 추출되므로 개선이 필요

In [17]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average',ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


- 분석: 쇼탱크 탈출처럼 유명한 영화가 처음 들어본듯한 영화보다 평점이 낮음 -> 왜곡된 평점 데이터를 회피할 방법이 필요
- **가중 평점(Weighted Rating) = (v/(v+m))*R + (m/(m+v))*C**
  - v: 개별 영화에 평점을 부여한 횟수
  - m: 평점을 부여하기 위한 최소 투표 횟수 
  - R: 개별 영화에 대한 평균 평점
  - C: 전체 영화에 대한 평균 평점

In [18]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('C:',round(C,3), 'm:', round(m,3))

C: 6.092 m: 370.2


In [19]:
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
  v = record['vote_count']
  R = record['vote_average']

  return (v/(v+m))*R + (m/(m+v))*C

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average,axis=1)

In [20]:
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote',ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


- **top_n의 2배수만큼 후보군으로 정한 뒤 weighted_vote 값이 높은 순으로 top_n만큼 추출하는 함수 만들기**



In [23]:
def find_sim_movie(df,sorted_ind,title_name,top_n=10):
  title_movie = df[df['title']==title_name]
  title_index = title_movie.index.values

  # top_n 2배에 해당하는 장르 유사성 높은 인덱스 추출하기
  similar_indexes = sorted_ind[title_index,:(top_n*2)]
  similar_indexes = similar_indexes.reshape(-1)
  # 기준 영화 인덱스는 제외하기
  similar_indexes = similar_indexes[similar_indexes != title_index]

  # weighted vote가 높은 순으로 top_n만큼 추출하기
  return df.iloc[similar_indexes].sort_values('weighted_vote',ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df,genre_sim_sorted_ind,'The Godfather',10)
similar_movies[['title','vote_average','weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
